This is the notebook to scrap the missing data from bitcoin dataset from kaggle.

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep

In [2]:
data=pd.read_csv('bitcoin.csv') 

In [3]:
nn_df=data[data.Open.isnull()]      #create mask for getting rows with null value

In [4]:
nn_df['dates'] = nn_df['Timestamp'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d')) #extract the dates for missing rows from unix time

<ipython-input-4-867de58fc6df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nn_df['dates'] = nn_df['Timestamp'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))


In [5]:
uniq_dates=list(nn_df['dates'].unique())

The dataset provided on kaggle has been abstracted from [bitcoincharts.com]('https://bitcoincharts.com/charts/bitstampUSD#rg60zczsg2021-04-07zeg2021-06-06ztgSzm1g10zm2g25zv'), this details is been provided in the description section of kaggle dataset. So we need to extract the from data from web site using webscraping and we will use Selenium to extract those data.

<img src='site.png'/>

On the above image , the is link with name `Load raw data`, it contains the value we required . We need to install [Selenium]('https://www.selenium.dev/') and [chrome web driver]('https://sites.google.com/a/chromium.org/chromedriver/') to access chrome for scraping data.

In [8]:
df=[] ##data would be stored in this list

In [22]:
for date in uniq_dates:
    driver = webdriver.Chrome(executable_path='')  ## put your path for chrome driver extension    
    url='https://bitcoincharts.com/charts/bitstampUSD#rg60zczsg'+date+'zeg'+date+'ztgSzm1g10zm2g25zv'
    driver.get(url)
    driver.find_element_by_partial_link_text("Load raw data").click()      #click on the `Load raw data`
    sleep(1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    mydiv=soup.find('table',{'id':'chart_table'})
    rows=mydiv.find('tbody').findAll('tr')
    try:
        for tr in rows:
            if(tr.findAll('td')[1].text!='—'):
                df.append([td.text.strip()  for td in tr.findAll('td')])
    except:
        pass
    sleep(1)
    driver.quit()

This will took around 10-11 hours to extract all required data.

In [23]:
len(df)

276866

We have extracted 276866 rows of data.

In [24]:
cp=df.copy()

In [25]:
cp[0]

['2011-12-30 10:45:00',
 '4.26',
 '4.26',
 '4.21',
 '4.21',
 '12.33',
 '51.96',
 '4.21']

In [26]:
import pickle

In [27]:
with open("bitcoincharts.txt", "wb") as fp:         ## saving data in text format
    pickle.dump(cp, fp)

In [28]:
bitcoin=pd.DataFrame(cp,columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price'])

In [29]:
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-30 10:45:00,4.26,4.26,4.21,4.21,12.33,51.96,4.21
1,2011-12-30 14:00:00,4.5,4.5,4.5,4.5,4.2,18.9,4.5
2,2011-12-30 15:15:00,4.21,4.21,4.21,4.21,39.77,167.43,4.21
3,2011-12-30 15:30:00,4.21,4.21,4.21,4.21,0.44,1.85,4.21
4,2011-12-31 07:45:00,4.39,4.39,4.39,4.39,0.46,2,4.39
...,...,...,...,...,...,...,...,...
276861,2021-03-30 23:00:00,58699.43,58734.72,58567.11,58733.62,7.85,460410.98,58628.91
276862,2021-03-30 23:15:00,58704.85,58748.45,58573.63,58748.45,5.69,334078.89,58662.7
276863,2021-03-30 23:30:00,58766.56,58766.56,58599.78,58600,24.67,1446170.82,58619.58
276864,2021-03-30 23:45:00,58599.99,58770.38,58599.99,58760.59,40.11,2353284.47,58669.39


In [30]:
bitcoin.to_csv('bitcoinchart.csv')

In [35]:
bitcoin_unix=pd.DataFrame(cp,columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price'])

In [36]:
bitcoin_unix

(276866, 8)

The date-time has been extracted in timestamp ,but we need to have unix time format in order to append these to previous data.

In [37]:
bitcoin_unix.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276866 entries, 0 to 276865
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Timestamp          276866 non-null  object
 1   Open               276866 non-null  object
 2   High               276866 non-null  object
 3   Low                276866 non-null  object
 4   Close              276866 non-null  object
 5   Volume_(BTC)       276866 non-null  object
 6   Volume_(Currency)  276866 non-null  object
 7   Weighted_Price     276866 non-null  object
dtypes: object(8)
memory usage: 16.9+ MB


In [66]:
bitcoin_unix['Timestamp']=(pd.to_datetime(bitcoin_unix.Timestamp)-pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')   ## converting date-time to unix format


In [68]:
bitcoin_unix.to_csv('bitcoinunix.csv')